# 每个LCS的数据划分并训练基础模型

In [1]:
# import module
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import torch
from SensorFormer.SensorFormer.many2many import *

2024-03-01 14:58:50.262514: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-01 14:58:50.305227: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 14:58:50.962409: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/zihao/anaconda3/envs/py11/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.2.0)/charset_normalizer (None) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or charde

In [2]:
mob_selected = [12, 16, 11, 10, 0] # selected LCS ordered by number of label samples

In [13]:
# 读取测试的传感器数据 #
num = 12
dat = pd.read_csv('/home/zihao/PycharmProjects/yucz/Nanjing_extended_data/my data/mob/mob'+ str(num) +'.csv')

In [157]:
from sklearn.model_selection import train_test_split
# Assuming df is your DataFrame and you want a 40% / 30% / 30% split

# (dat.time > '2022-07-01') & (dat.time < '2022-10-01')
# (dat.time > '2022-04-01') & (dat.time < '2022-7-01')
test_dat = dat[(dat.time > '2022-10-01')] 
# test_dat = dat[(dat.time > '2022-04-01') & (dat.time < '2022-7-01')] 
# test_dat = dat[(dat.time > '2022-07-01') & (dat.time < '2022-10-01')]

# First split: Separate out the training set
df_train, df_temp = train_test_split(test_dat, train_size=0.4, shuffle=False)

# Second split: Separate out the testing and validation sets
df_val, df_test = train_test_split(df_temp, test_size=0.5, shuffle=False)

print('train: label samples:', df_train[df_train.ref_pm2d5 > 0].shape[0], 'total samples:', df_train.shape[0])
print('validation: label samples:', df_val[df_val.ref_pm2d5 > 0].shape[0], 'total samples:', df_val.shape[0])
print('test: label samples:', df_test[df_test.ref_pm2d5 > 0].shape[0], 'total samples:', df_test.shape[0])

train: label samples: 6310 total samples: 31114
validation: label samples: 1526 total samples: 10372
test: label samples: 2265 total samples: 10372


In [174]:
# 按月切割多个传感器数据 #
for i, num in enumerate([12, 16, 11, 10, 0]):
    dat = pd.read_csv('/home/zihao/PycharmProjects/yucz/Nanjing_extended_data/my data/mob/mob'+ str(num) +'.csv')
    dat['time'] = pd.to_datetime(dat['time'])
    for month in range(4, 12):
        # 构建当前月份的起始日期和结束日期
        current_start_date = pd.to_datetime(f'2022-{month:02d}-01')
        current_end_date = pd.to_datetime(f'2022-{month:02d}-01') + pd.offsets.MonthEnd(1)
        
        test_dat = dat[(dat.time > current_start_date) & (dat.time < current_end_date)] 
    

        # First split: Separate out the training set
        df_train, df_temp = train_test_split(test_dat, train_size=0.4, shuffle=False)

        # Second split: Separate out the testing and validation sets
        df_val, df_test = train_test_split(df_temp, test_size=0.5, shuffle=False)

        df_train.to_pickle('./my data/train/train'+ str(num)+ '_' + str(month)+'.pkl')
        df_val.to_pickle('./my data/val/val'+ str(num)+ '_' + str(month)+'.pkl')
        df_test.to_pickle('./my data/test/test'+ str(num)+ '_' + str(month)+'.pkl')

       

In [176]:
# 检验切割后的数据情况 #
for i, num in enumerate([12, 16, 11, 10, 0]):
    for month in range(4, 12):
        df_train = pd.read_pickle('./my data/train/train'+ str(num)+ '_' + str(month)+'.pkl')
        df_val = pd.read_pickle('./my data/val/val'+ str(num)+ '_' + str(month)+'.pkl')
        df_test = pd.read_pickle('./my data/test/test'+ str(num)+ '_' + str(month)+'.pkl')
        print(num, month, df_train[df_train.ref_pm2d5 > 0].shape[0], df_val[df_val.ref_pm2d5 > 0].shape[0], df_test[df_test.ref_pm2d5 > 0].shape[0])
        

12 4 1607 1528 1409
12 5 487 243 970
12 6 1778 1387 1222
12 7 1396 1044 1027
12 8 2144 2177 1297
12 9 1420 982 825
12 10 2095 1646 1255
12 11 1874 1462 1329
16 4 743 421 386
16 5 188 0 243
16 6 243 131 280
16 7 0 0 0
16 8 303 226 282
16 9 561 275 534
16 10 778 322 474
16 11 736 636 842
11 4 188 237 480
11 5 486 224 1024
11 6 393 176 0
11 7 0 164 386
11 8 682 339 370
11 9 375 387 275
11 10 207 286 190
11 11 43 159 128
10 4 635 286 0
10 5 0 451 276
10 6 429 144 274
10 7 496 87 176
10 8 342 175 444
10 9 293 328 174
10 10 730 184 134
10 11 408 411 126
0 4 258 212 192
0 5 346 239 0
0 6 251 229 203
0 7 228 197 185
0 8 308 230 249
0 9 467 110 28
0 10 282 200 216
0 11 279 185 189


In [130]:
# 计算分段的MAE #
# 4~6: 11.49(11.83)(8.96 w11), 7~9: 5.15(9.42)(5.67 w11), 10~11: 6.22 (13.68)(6.80 w11)#

print('Test MAE: ', mean_absolute_error(df_test[df_test.ref_pm2d5 > 0].pm2d5, df_test[df_test.ref_pm2d5 > 0].ref_pm2d5))

# 4~6: 6.16(8.91), 7~9: 6.83(9.11), 10~11:  11.68(17.76)#
print('Validation MAE: ', mean_absolute_error(df_val[df_val.ref_pm2d5 > 0].pm2d5, df_val[df_val.ref_pm2d5 > 0].ref_pm2d5))

Test MAE:  9.416035623720171
Validation MAE:  9.118129587243674


In [107]:
df_train.to_pickle('./my data/train/train12_4to6.pkl')
df_val.to_pickle('./my data/val/val12_4to6.pkl')
df_test.to_pickle('./my data/test/test12_4to6.pkl')


In [166]:
feature_names = ['pm2d5', 'hmd', 'tmp', 'weight']
target_names =['ref_pm2d5']

# test_dat = df_test
test_dat = pd.read_csv('/home/zihao/PycharmProjects/yucz/Nanjing_extended_data/my data/mob/mob'+ str(11) +'.csv')

# 将读取的数据转换成列表
test_features = test_dat[feature_names].values.astype(np.float32)
test_target = test_dat[target_names].values.astype(np.float32)

# 切割数据进入data loader，以进行inference
test_dataset = TimeseriesDataset(test_features, test_target, seq_len=12)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle = False)

In [167]:
# Inference w mob11 training model#
device = torch.device("cuda")

best_model = torch.load('SensorFormer/SensorFormer/result/CLfor12/AdamW_lr0.0003_ep50_4to6.pt')
pred,test_loss = test(best_model, device, test_loader)
pred = np.array(pred)
pred_df = all_pred(pred,12)
pred_df = pred_df.apply(lambda row: row.fillna(row.mean()), axis=1)
pred_df['mean'] = pred_df.iloc[:, 0:].mean(axis=1)
    
pred_true = test_target[:pred_df.shape[0],0]
pred_df['true'] = pred_true

# inverse_pred = scaler_y.inverse_transform(pred_df['mean'].values.reshape(-1,1))
# inverse_true = scaler_y.inverse_transform(pred_true.reshape(-1,1))
valid_pred = pred_df[pred_df['true'] != 0]

pred_df.to_csv('SensorFormer/SensorFormer/result/CLfor12/test_10to11_w11.csv')

print('Test MAE: ', mean_absolute_error(valid_pred.true, valid_pred['mean']))

sum of test loss:  60625.11328125 
valid_cnt of test set:  89244.0
Test set: Average loss: 0.6793
Test MAE:  8.14988416408489


# Train

In [9]:
from torch.utils.tensorboard import SummaryWriter
from SensorFormer.SensorFormer.many2manyCopy import *

torch.manual_seed(1)

device = torch.device("cuda")

train_kwargs = {'batch_size': 8}
test_kwargs = {'batch_size': 1}
cuda_kwargs = {'num_workers': 1,
                'pin_memory': True,
                'shuffle': False}
train_kwargs.update(cuda_kwargs)
test_kwargs.update(cuda_kwargs)

prev = [12, 8] # 便于后续索引上一个模型

for month in range(8, 12):
    for i, num in enumerate([12, 16, 11, 10, 0]):
        df_train = pd.read_pickle('./my data/train/train'+ str(num)+ '_' + str(month)+'.pkl')
        df_val = pd.read_pickle('./my data/val/val'+ str(num)+ '_' + str(month)+'.pkl')
        df_test = pd.read_pickle('./my data/test/test'+ str(num)+ '_' + str(month)+'.pkl')

        train_features = df_train[feature_names].values.astype(np.float32)
        train_target = df_train[target_names].values.astype(np.float32)
        val_features = df_val[feature_names].values.astype(np.float32)
        val_target = df_val[target_names].values.astype(np.float32)
        test_features = df_test[feature_names].values.astype(np.float32)
        test_target = df_test[target_names].values.astype(np.float32)


        train_dataset = TimeseriesDataset(train_features, train_target, seq_len=12)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 8, shuffle = False)
        val_dataset = TimeseriesDataset(val_features, val_target, seq_len=12)
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 1, shuffle = False)   
        test_dataset = TimeseriesDataset(test_features, test_target, seq_len=12)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle = False)

        
        model_pth = 'SensorFormer/SensorFormer/result/ensemble/LCS' + str(num)+ '_' + str(month)+'.pt'

        model = Net().to(device)
        optimizer = optim.AdamW(model.parameters(), lr=0.0003) # optimizer = optim.Adadelta(model.parameters(), lr=args.lr) # 0.0001的学习率下效果奇差

        if num != 12 or month != 8:
            prev_pth = 'SensorFormer/SensorFormer/result/ensemble/LCS' + str(prev[0])+ '_' + str(prev[1])+'.pt'
            model_prev = torch.load(prev_pth)  # 加载上一个模型
            model.load_state_dict(model_prev['model_state_dict'])  # 加载模型可学习参数
            optimizer.load_state_dict(model_prev['optimizer_state_dict'])  # 加载优化器参数
            print("\nprev_model loaded: " + prev_pth)
        


        writer = SummaryWriter('runs/Ensemble_' + str(num)+ '_' + str(month))

        best_val = 100
        # best_inverse = 100 
        best_epoch = 0
        
        start_epoch = 1
        end_epoch = 25

        for epoch in range(start_epoch , end_epoch):
            #scheduler.step()

            train(model, device, train_loader, optimizer, epoch, writer) 
            
            

            ### get the validation MAE and save the current best model!
            pred, test_loss = test(model, device, val_loader)
            
            pred = np.array(pred)
    
            pred_df = all_pred(pred,12)
            pred_df = pred_df.apply(lambda row: row.fillna(row.mean()), axis=1)
            pred_df['mean'] = pred_df.iloc[:, 0:].mean(axis=1)
            
            pred_true = val_target[:pred_df.shape[0],0]
            pred_df['true'] = pred_true
            valid_pred = pred_df[pred_df['true'] != 0]
            val_mae = (mean_absolute_error(valid_pred.true, valid_pred['mean'])) # 不是所有数据都能加入平均，得看权重

            writer.add_scalar("Loss/val_mae", test_loss, epoch) 
            writer.flush()

            #val_mae = test_loss
            print('val mae: ', val_mae, '\n')
            if val_mae < best_val:
                print('renew best model and save...',val_mae)
                best_val = val_mae
                # best_inverse = inverse_val
                best_epoch = epoch
                
                
                pred_df.to_csv('SensorFormer/SensorFormer/result/ensemble/LCS' + str(num)+ '_' + str(month)+ 'val.csv',index=False)


                # torch.save(model, model_pth)
                state = {
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }
                
                torch.save(state, model_pth)
            

            
        ## now use the found best model to predict the test data
        # best_model = torch.load(model_pth + ".pt")
        best_model = torch.load(model_pth)  # 加载当前最佳
        model.load_state_dict(best_model['model_state_dict'])  # 加载模型可学习参数

        pred,test_loss = test(model, device, test_loader)
        pred = np.array(pred)
        pred_df = all_pred(pred,12)
        pred_df = pred_df.apply(lambda row: row.fillna(row.mean()), axis=1)
        pred_df['mean'] = pred_df.iloc[:, 0:].mean(axis=1)
            
        pred_true = test_target[:pred_df.shape[0],0]
        pred_df['true'] = pred_true
        valid_pred = pred_df[pred_df['true'] != 0]
        
        
        pred_df.to_csv('SensorFormer/SensorFormer/result/ensemble/LCS' + str(num)+ '_' + str(month)+ 'test.csv', index=False)
        
        print('Best Val MAE: ', best_val)
        print('Best Val Epoch: ', best_epoch)
        print('Test MAE: ',mean_absolute_error(valid_pred.true, valid_pred['mean'])) # 不是所有数据都能加入平均，得看权重

        writer.close()
        prev = [num, month] # 更新prev便于后续索引上一个模型

Train Epoch: 1 [5600/8850 (63%)]	Loss: 4.219543
Train Epoch: 1 [7200/8850 (81%)]	Loss: 7.764049
sum of train loss:  25531.466796875 
valid cnt of train set:  25728.0
avg loss of train set:  0.9923611161720693
sum of test loss:  33399.84765625 
valid_cnt of test set:  26124.0
Test set: Average loss: 1.2785
val mae:  15.342142600849638 

renew best model and save... 15.342142600849638
Train Epoch: 2 [5600/8850 (63%)]	Loss: 3.206766
Train Epoch: 2 [7200/8850 (81%)]	Loss: 5.632327
sum of train loss:  18950.357421875 
valid cnt of train set:  25728.0
avg loss of train set:  0.7365655092457634
sum of test loss:  28708.4375 
valid_cnt of test set:  26124.0
Test set: Average loss: 1.0989
val mae:  13.186513620940818 

renew best model and save... 13.186513620940818
Train Epoch: 3 [5600/8850 (63%)]	Loss: 2.378612
Train Epoch: 3 [7200/8850 (81%)]	Loss: 3.745052
sum of train loss:  15788.072265625 
valid cnt of train set:  25728.0
avg loss of train set:  0.6136533063442553
sum of test loss:  2450

In [12]:
# 

for i, num in enumerate([12, 16, 11, 10, 0]):
    for month in range(8, 12):
    
        df = pd.read_csv('SensorFormer/SensorFormer/result/ensemble/LCS' + str(num)+ '_' + str(month) +'val.csv')
        df_test = df[df.true > 0]
        mae = mean_absolute_error(df_test['mean'], df_test['true'])
        rec_test.append(mae)
        print(str(num)+ '_' + str(month) + '_Test MAE: ', mae)

12_8_Test MAE:  7.823127374362822
12_9_Test MAE:  9.1672661708196
12_10_Test MAE:  6.786005112155857
12_11_Test MAE:  6.700255340981806
16_8_Test MAE:  6.553765770621333
16_9_Test MAE:  5.344732676090013
16_10_Test MAE:  3.552074621445197
16_11_Test MAE:  9.886001308436997
11_8_Test MAE:  6.519017148507561
11_9_Test MAE:  4.694452062294941
11_10_Test MAE:  3.787962616502193
11_11_Test MAE:  13.051935476601123
10_8_Test MAE:  19.747932843244115
10_9_Test MAE:  9.76491844926666
10_10_Test MAE:  6.934761581378196
10_11_Test MAE:  8.033874794601763
0_8_Test MAE:  4.8387946220995435
0_9_Test MAE:  4.227953692731402
0_10_Test MAE:  7.30342842757849
0_11_Test MAE:  25.407177771238633


In [13]:
from torch.utils.tensorboard import SummaryWriter
from SensorFormer.SensorFormer.many2manyCopy import *

torch.manual_seed(1)

device = torch.device("cuda")

train_kwargs = {'batch_size': 8}
test_kwargs = {'batch_size': 1}
cuda_kwargs = {'num_workers': 1,
                'pin_memory': True,
                'shuffle': False}
train_kwargs.update(cuda_kwargs)
test_kwargs.update(cuda_kwargs)


for month in range(8, 12):
    for i, num in enumerate([12, 16, 11, 10, 0]):
        df_train = pd.read_pickle('./my data/train/train'+ str(num)+ '_' + str(month)+'.pkl')
        df_val = pd.read_pickle('./my data/val/val'+ str(num)+ '_' + str(month)+'.pkl')
        df_test = pd.read_pickle('./my data/test/test'+ str(num)+ '_' + str(month)+'.pkl')

        train_features = df_train[feature_names].values.astype(np.float32)
        train_target = df_train[target_names].values.astype(np.float32)
        val_features = df_val[feature_names].values.astype(np.float32)
        val_target = df_val[target_names].values.astype(np.float32)
        test_features = df_test[feature_names].values.astype(np.float32)
        test_target = df_test[target_names].values.astype(np.float32)


        train_dataset = TimeseriesDataset(train_features, train_target, seq_len=12)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 8, shuffle = False)
        val_dataset = TimeseriesDataset(val_features, val_target, seq_len=12)
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 1, shuffle = False)   
        test_dataset = TimeseriesDataset(test_features, test_target, seq_len=12)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle = False)

        
        model_pth = 'SensorFormer/SensorFormer/result/individual/LCS' + str(num)+ '_' + str(month)+'.pt'

        model = Net().to(device)
        optimizer = optim.AdamW(model.parameters(), lr=0.0003) # optimizer = optim.Adadelta(model.parameters(), lr=args.lr) # 0.0001的学习率下效果奇差
        

        writer = SummaryWriter('runs/ind_' + str(num)+ '_' + str(month))

        best_val = 100
        # best_inverse = 100 
        best_epoch = 0
        
        start_epoch = 1
        end_epoch = 25

        for epoch in range(start_epoch , end_epoch):
            #scheduler.step()

            train(model, device, train_loader, optimizer, epoch, writer) 
            
            

            ### get the validation MAE and save the current best model!
            pred, test_loss = test(model, device, val_loader)
            
            pred = np.array(pred)
    
            pred_df = all_pred(pred,12)
            pred_df = pred_df.apply(lambda row: row.fillna(row.mean()), axis=1)
            pred_df['mean'] = pred_df.iloc[:, 0:].mean(axis=1)
            
            pred_true = val_target[:pred_df.shape[0],0]
            pred_df['true'] = pred_true
            valid_pred = pred_df[pred_df['true'] != 0]
            val_mae = (mean_absolute_error(valid_pred.true, valid_pred['mean'])) # 不是所有数据都能加入平均，得看权重

            writer.add_scalar("Loss/val", test_loss, epoch) 
            writer.flush()

            #val_mae = test_loss
            print('val mae: ', val_mae, '\n')
            if val_mae < best_val:
                print('renew best model and save...',val_mae)
                best_val = val_mae
                # best_inverse = inverse_val
                best_epoch = epoch
                
                
                pred_df.to_csv('SensorFormer/SensorFormer/result/individual/LCS' + str(num)+ '_' + str(month)+ 'val.csv',index=False)


                # torch.save(model, model_pth)
                state = {
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }
                
                torch.save(state, model_pth)
            

            
        ## now use the found best model to predict the test data
        # best_model = torch.load(model_pth + ".pt")
        best_model = torch.load(model_pth)  # 加载当前最佳
        model.load_state_dict(best_model['model_state_dict'])  # 加载模型可学习参数

        pred,test_loss = test(model, device, test_loader)
        pred = np.array(pred)
        pred_df = all_pred(pred,12)
        pred_df = pred_df.apply(lambda row: row.fillna(row.mean()), axis=1)
        pred_df['mean'] = pred_df.iloc[:, 0:].mean(axis=1)
            
        pred_true = test_target[:pred_df.shape[0],0]
        pred_df['true'] = pred_true
        valid_pred = pred_df[pred_df['true'] != 0]
        
        
        pred_df.to_csv('SensorFormer/SensorFormer/result/individual/LCS' + str(num)+ '_' + str(month)+ 'test.csv', index=False)
        
        print('Best Val MAE: ', best_val)
        print('Best Val Epoch: ', best_epoch)
        print('Test MAE: ',mean_absolute_error(valid_pred.true, valid_pred['mean'])) # 不是所有数据都能加入平均，得看权重

        writer.close()

Train Epoch: 1 [5600/8850 (63%)]	Loss: 4.219543
Train Epoch: 1 [7200/8850 (81%)]	Loss: 7.764049
sum of train loss:  25531.466796875 
valid cnt of train set:  25728.0
avg loss of train set:  0.9923611161720693
sum of test loss:  33399.84765625 
valid_cnt of test set:  26124.0
Test set: Average loss: 1.2785
val mae:  15.342142600849638 

renew best model and save... 15.342142600849638
Train Epoch: 2 [5600/8850 (63%)]	Loss: 3.206766
Train Epoch: 2 [7200/8850 (81%)]	Loss: 5.632327
sum of train loss:  18950.357421875 
valid cnt of train set:  25728.0
avg loss of train set:  0.7365655092457634
sum of test loss:  28708.4375 
valid_cnt of test set:  26124.0
Test set: Average loss: 1.0989
val mae:  13.186513620940818 

renew best model and save... 13.186513620940818
Train Epoch: 3 [5600/8850 (63%)]	Loss: 2.378612
Train Epoch: 3 [7200/8850 (81%)]	Loss: 3.745052
sum of train loss:  15788.072265625 
valid cnt of train set:  25728.0
avg loss of train set:  0.6136533063442553
sum of test loss:  2450

In [18]:
rec_test_en = [[] for i in range(5)]
rec_test_ind = [[] for i in range(5)]
for i, num in enumerate([12, 16, 11, 10, 0]):
    for month in range(8, 12):
        df = pd.read_csv('SensorFormer/SensorFormer/result/ensemble/LCS' + str(num)+ '_' + str(month) +'val.csv')
        df_test = df[df.true > 0]
        mae = mean_absolute_error(df_test['mean'], df_test['true'])
        rec_test_en[i].append(mae)
        print('ensemble_'+str(num)+ '_' + str(month) + '_Test MAE: ', mae)
        df = pd.read_csv('SensorFormer/SensorFormer/result/individual/LCS' + str(num)+ '_' + str(month) +'val.csv')
        df_test = df[df.true > 0]
        mae = mean_absolute_error(df_test['mean'], df_test['true'])
        rec_test_ind[i].append(mae)
        print('individual_'+ str(num)+ '_' + str(month) + '_Test MAE: ', mae)

ensemble_12_8_Test MAE:  7.823127374362822
individual_12_8_Test MAE:  6.709114417271343
ensemble_12_9_Test MAE:  9.1672661708196
individual_12_9_Test MAE:  4.060094990328228
ensemble_12_10_Test MAE:  6.786005112155857
individual_12_10_Test MAE:  5.654731968199497
ensemble_12_11_Test MAE:  6.700255340981806
individual_12_11_Test MAE:  6.3071693011865975
ensemble_16_8_Test MAE:  6.553765770621333
individual_16_8_Test MAE:  6.969539018116167
ensemble_16_9_Test MAE:  5.344732676090013
individual_16_9_Test MAE:  3.4048804590965043
ensemble_16_10_Test MAE:  3.552074621445197
individual_16_10_Test MAE:  5.280608429063675
ensemble_16_11_Test MAE:  9.886001308436997
individual_16_11_Test MAE:  8.598578304230792
ensemble_11_8_Test MAE:  6.519017148507561
individual_11_8_Test MAE:  7.305176657411319
ensemble_11_9_Test MAE:  4.694452062294941
individual_11_9_Test MAE:  7.2461097697817385
ensemble_11_10_Test MAE:  3.787962616502193
individual_11_10_Test MAE:  7.9634765787057935
ensemble_11_11_Test 

In [21]:
rec_test_ind

[[6.709114417271343, 4.060094990328228, 5.654731968199497, 6.3071693011865975],
 [6.969539018116167, 3.4048804590965043, 5.280608429063675, 8.598578304230792],
 [7.305176657411319,
  7.2461097697817385,
  7.9634765787057935,
  19.259359835601703],
 [4.782929410393668, 3.2825184720240954, 4.715017103274663, 11.44801540974186],
 [4.238846122044298, 4.1447456578168005, 8.278195781804584, 24.57390784975344]]

In [27]:
from plotly import graph_objects as go
from plotly_resampler import register_plotly_resampler

register_plotly_resampler(mode='auto')

fig = go.Figure()

## 描出所有resample数据

for i, num in enumerate([12, 16, 11, 10, 0]):
        fig.add_trace(  
                dict(name="en_mob#" + str(num), 
                mode='markers+lines',
                showlegend=True),  ## 没有的话legend不显示       
                hf_x=[8, 9, 10, 11],
                hf_y=rec_test_en[i])
        fig.add_trace(  
                dict(name="ind_mob#" + str(num), 
                mode='markers+lines',
                showlegend=True),  ## 没有的话legend不显示       
                hf_x=[8, 9, 10, 11],
                hf_y=rec_test_ind[i])

fig.update_layout(
            title_text="ens. vs ind.", #Add a chart title·
            title_font_family="Times New Roman",
            title_font_size = 20,
            title_font_color="darkblue", #Specify font color of the title
            title_x=0.5, #Specify the title position
            yaxis_title='MAE',
            width=800,
            # legend=dict(orientation="h", y=1.1, xanchor="left", x=0)
            )


fig

FigureWidgetResampler({
    'data': [{'mode': 'markers+lines',
              'name': 'en_mob#12',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'de1c09cf-380f-4710-a955-963169e8a4ea',
              'x': array([ 8,  9, 10, 11]),
              'y': array([7.82312737, 9.16726617, 6.78600511, 6.70025534])},
             {'mode': 'markers+lines',
              'name': 'ind_mob#12',
              'showlegend': True,
              'type': 'scatter',
              'uid': '0f16e251-26b9-4035-a7d8-e8434a191b9a',
              'x': array([ 8,  9, 10, 11]),
              'y': array([6.70911442, 4.06009499, 5.65473197, 6.3071693 ])},
             {'mode': 'markers+lines',
              'name': 'en_mob#16',
              'showlegend': True,
              'type': 'scatter',
              'uid': '8935451f-cd37-4822-833e-1b3776b568ce',
              'x': array([ 8,  9, 10, 11]),
              'y': array([6.55376577, 5.34473268, 3.55207462, 9.88600131])},
      

# 评估 inference 效果

In [30]:
# 加载需要评估的数据 #
import pandas as pd
num, month = 0, 11
res = pd.read_csv("SensorFormer/SensorFormer/result/ensemble/LCS" + str(num)+ '_' + str(month)+"val.csv")
res


,0,1,2,3,4,5,6,7,8,9,10,11,mean,true
0,20.653503,20.653503,20.653503,20.653503,20.653503,20.653503,20.653503,20.653503,20.653503,20.653503,20.653503,20.653503,20.653503,0.0
1,20.546967,20.572966,20.559966,20.559966,20.559966,20.559966,20.559966,20.559966,20.559966,20.559966,20.559966,20.559966,20.559966,0.0
2,20.413076,20.465168,20.561453,20.479899,20.479899,20.479899,20.479899,20.479899,20.479899,20.479899,20.479899,20.479899,20.479899,0.0
3,20.309454,20.333029,20.455427,20.550531,20.412110,20.412110,20.412110,20.412110,20.412110,20.412110,20.412110,20.412110,20.412110,0.0
4,20.396967,20.383520,20.477430,20.598545,21.861151,20.743523,20.743523,20.743523,20.743523,20.743523,20.743523,20.743523,20.743523,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3218,9.471735,9.289940,9.146330,9.132939,9.248244,9.359930,9.360800,9.550306,9.400845,9.329959,9.383850,9.502398,9.348106,0.0
3219,9.280585,9.134786,9.069913,9.133585,9.246998,9.284830,9.508913,9.356386,9.243697,9.258433,9.385912,9.501174,9.283768,0.0
3220,9.160464,9.093537,9.104270,9.165529,9.204374,9.467074,9.349357,9.233972,9.206485,9.294152,9.417817,9.423157,9.260016,0.0
3221,9.224580,9.230919,9.239669,9.228724,9.492744,9.414259,9.333737,9.301317,9.345841,9.429508,9.445715,10.391315,9.423194,0.0


In [33]:
df = pd.read_pickle('./my data/test/test'+ str(num)+ '_' + str(month)+'.pkl')
df = df_test[:-11]
df 

,0,1,2,3,4,5,6,7,8,9,10,11,mean,true
64,24.540226,24.492502,24.467781,24.468021,24.482325,24.487246,24.480185,24.467331,23.663067,23.644699,23.612492,23.562050,24.197327,40.760
65,24.478634,24.451633,24.450466,24.461740,24.470791,24.464512,24.454411,23.647898,23.627296,23.591879,23.546791,23.500416,24.095539,40.760
69,24.551281,24.542389,24.532719,23.732471,23.717421,23.680683,23.629902,23.580715,23.554985,23.555084,23.569403,23.573116,23.851681,42.326
70,24.464817,24.452915,23.650700,23.633167,23.601093,23.551428,23.504616,23.476076,23.473913,23.485075,23.494129,23.486588,23.689543,42.326
92,22.773455,22.761772,22.484003,22.440758,22.460526,22.464277,22.455223,22.441174,22.035297,22.017208,22.470892,22.456173,22.438396,43.129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2896,31.265604,31.252817,31.351669,31.659687,31.651339,31.637060,31.607033,32.287964,32.274792,32.253635,32.221928,32.175636,31.803264,8.179
2897,31.257416,31.356287,31.665226,31.648903,31.633970,31.601883,32.288025,32.277157,32.256618,32.218403,32.173817,32.131596,31.875775,8.179
2898,31.392212,31.705915,31.688770,31.665695,31.632936,32.316559,32.311584,32.293362,32.255753,32.204662,32.164173,32.142677,31.981192,8.179
2991,31.360680,31.320850,31.266695,31.217527,31.196743,31.199789,31.215805,31.221153,31.213612,31.202774,27.760447,27.122391,30.608205,23.936


In [165]:
from plotly import graph_objects as go
from plotly_resampler import register_plotly_resampler

register_plotly_resampler(mode='auto')

fig = go.Figure()

## 描出所有resample数据

fig.add_trace(  
        dict(name="ref mob#" + str(num), 
        mode='lines',
        showlegend=True),  ## 没有的话legend不显示       
        hf_x=df[df.ref_pm2d5 > 0].time,
        hf_y=res[res.true > 0]['true'])

fig.add_trace(  
        dict(name="cal mob#" + str(num), 
        mode='lines',
        showlegend=True),  ## 没有的话legend不显示       
        hf_x=df.time,
        hf_y=res['mean'])

fig.add_trace(  
        dict(name="ori mob#" + str(num), 
        mode='lines',
        showlegend=True),  ## 没有的话legend不显示       
        hf_x=df.time,
        hf_y=df.pm2d5)

fig.update_layout(
            title_text="calibrated values vs ref values", #Add a chart title·
            title_font_family="Times New Roman",
            title_font_size = 20,
            title_font_color="darkblue", #Specify font color of the title
            title_x=0.5, #Specify the title position
            yaxis_title='PM2.5 concentration',
            # legend=dict(orientation="h", y=1.1, xanchor="left", x=0)
            )


fig

FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': '<b style="color:sandybrown">[R]</b> ref mob#12 <i style="color:#fc9944">~13m</i>',
              'showlegend': True,
              'type': 'scatter',
              'uid': '212ca030-fa69-4991-a01a-acd68dedfcf9',
              'x': array([datetime.datetime(2022, 11, 20, 14, 50),
                          datetime.datetime(2022, 11, 20, 14, 51),
                          datetime.datetime(2022, 11, 20, 14, 53), ...,
                          datetime.datetime(2022, 11, 30, 18, 53),
                          datetime.datetime(2022, 11, 30, 18, 55),
                          datetime.datetime(2022, 11, 30, 18, 57)], dtype=object),
              'y': array([25.552, 25.552, 25.552, ..., 14.67 , 14.67 , 14.754])},
             {'mode': 'lines',
              'name': '<b style="color:sandybrown">[R]</b> cal mob#12 <i style="color:#fc9944">~15m</i>',
              'showlegend': True,
              'type': 'scatter',
  

# NN

In [ ]:
class RegressionNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DeeperRegressionNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

# Define the input size, hidden size, and output size
input_size = 10
hidden_size = 20
output_size = 1

# Create an instance of the deeper regression network
model = DeeperRegressionNet(input_size, hidden_size, output_size)

# Print the model architecture
print(model)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your model architecture
class DeepRegressor(nn.Module):
    def __init__(self):
        super(DeepRegressor, self).__init__()
        self.fc1 = nn.Linear(in_features=10, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Prepare your dataset and dataloaders

# Initialize your model
model = DeepRegressor()

# Choose a loss function
criterion = nn.MSELoss()

# Choose an optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train your model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in train_dataloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        running_loss += loss.item()

    # Print the average loss for this epoch
    print(f"Epoch {epoch+1} - Loss: {running_loss / len(train_dataloader)}")

# Evaluate your model on the validation set

# Make predictions on test data

